In [ ]:
import os
import sys
from step_function_utils import send_task_success
from video_processor import ffmpeg_check, extract_frames
from utils import download_file, parse_location, upload_file
from get_image_embeddings import get_images_embeddings
from similarity import cosine_similarity_list, filter_relevant_frames

tmp_path                    = "./tmp"
difference_threshold        = 0.9

os.environ['S3_URI'] = 's3://you-bucket/video_in/video_corto.mp4'


In [ ]:
s3_uri = os.environ.get("S3_URI", "s3://bucket/key")
task_token = os.environ.get("TASK_TOKEN", None)

print(s3_uri)

In [ ]:
#ffmpeg_check()

# Parse the S3 URI
bucket, prefix, fileName, extension, file  = parse_location(s3_uri)

# Print bucket and key
print(f"Bucket: {bucket}")
print(f"prefix: {prefix}")
print(f"extension: {extension}")
print(f"file: {file}")

In [ ]:
local_path              = f"{tmp_path}/{file}"
location                = f"{prefix}/{file}"
output_dir              = f"{tmp_path}/{fileName}"

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(local_path), exist_ok=True)

print(f"descargando {file} s3://{bucket}/{prefix} to {local_path}")
download_file(bucket,location, local_path)

In [ ]:
files = extract_frames(local_path, output_dir)

In [ ]:
embed_1024 = get_images_embeddings(files, embedding_dimmesion=1024)

In [ ]:
similarity_1024 = cosine_similarity_list(embed_1024)

In [ ]:
similarity_1024.append(0.5) # add this so the last one is pick
selected_frames = filter_relevant_frames( similarity_1024, difference_threshold = difference_threshold)

In [ ]:
selected_frames

In [ ]:

selected_frames_real = []

for sf in selected_frames:

    origen_file = f"{output_dir}/sec_{str(sf+1).zfill(5)}.jpg"
    real_frame = sf + 1
    destination_key = f"{prefix}/selected_frames/{real_frame}.jpg"

    print(f"{origen_file} => {destination_key}")
    upload_file(bucket, destination_key, origen_file)
    selected_frames_real.append(real_frame)
